1. 문서 내용 읽기  
2. 문서 분할(쪼개기)  
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음.  
        - 문서 길이(input)가 길어서 답변 생성에 시간 소요됨 - > 답변 늦어짐  
3. 임베딩 - 벡터 데이터베이스에 저장
4. 질문이 있으면, 벡터 데이터베이스 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달



In [ ]:
## 패키지/라이브러리 설치
# %pip install -qU docx2txt langchain-community langchain-text-splitters langchain-chroma

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader


## 1. 문서 내용 읽고 분할 #############################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 벡터 데이터베이스에 저장 #############################
   
## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')


In [ ]:

## 2.3. 벡터 데이터베이스에 저장  

## 데이터베이스에 저장
##[방법1] in memory
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
# )



In [ ]:
# %pip install langchain-pinecone

  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.12.6
    Uninstalling aiohttp-3.12.6:
      Successfully uninstalled aiohttp-3.12.6
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [10]:
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

In [ ]:
from langchain_pinecone import PineconeVectorStore

database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name='law-index',
)

database

In [14]:
len(document_list)

23

In [13]:

## 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색  

## 3.1. 사용자 질문
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query)

## 4.0 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 #########

## 4.1. 프롬프트 작성

prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''
## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query,)

## 4.3. LLM 모델 생성(chatOpenAI 인스턴스 생성)  
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서를 보냄  
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='전세사기피해자에 대한 금융지원은 다음과 같은 방식으로 이루어집니다:\n\n1. **융자 지원**: 국가 및 지방자치단체는 전세사기피해자 및 특정 요건을 갖춘 임차인에게 긴급한 주거안정을 보호하기 위해 필요한 자금을 융자할 수 있습니다. 이는 피해자들이 주거 문제로 인한 금융 부담을 줄이고 안정적인 생활을 유지할 수 있도록 도와줍니다.\n\n2. **우선변제 미수령자 지원**: 주택도시기금에서 주택 임대차에 필요한 자금을 융자할 수 있으며, 이 경우 「주택임대차보호법」에 따라 우선변제를 받지 못한 전세사기피해자가 시급한 지원이 필요하다고 인정되는 경우에 적용됩니다.\n\n3. **신용 정보 보호**: 전세사기피해자의 대출에 관련하여 모든 보증금이 변제되지 않은 경우, 신용정보집중관리위원회를 통해 해당 대출에 대한 채무 불이행 및 대위변제의 등록을 유예하거나 삭제할 수 있습니다. 이는 피해자가 경제적으로 다시 설 수 있도록 신용 회복을 지원하는 조치입니다.\n\n4. **주택저당채권대상 확대**: 「한국주택금융공사법」에 따른 주택저당채권대상에 주거 목적으로 사용되는 오피스텔도 포함되어, 전세사기피해자 등이 보다 다양한 주거 옵션을 마련할 수 있도록 지원됩니다.\n\n이러한 금융지원 방안은 전세사기피해자가 겪는 금융적 부담을 완화하고 주거 안정성을 확보하기 위해 마련된 조치들입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 381, 'prompt_tokens': 4160, 'total_tokens': 4541, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'a

In [15]:
retrieved_docs

[Document(id='5388f660-5158-4747-ad60-3e394d784ee0', metadata={'source': 'law_1.docx'}, page_content='전세사기피해자 지원 및 주거안정에 관한 특별법 ( 약칭: 전세사기피해자법 )\n\n[시행 2024. 11. 11.] [법률 제20429호, 2024. 9. 10., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n       제1장 총칙 \n\n제1조(목적) 이 법은 전세사기로 피해를 입은 임차인에게 경ㆍ공매 절차 및 조세 징수 등에 관한 특례를 부여함으로써 전세사기피해자를 지원하고 주거안정을 도모함을 목적으로 한다.\n\n제2조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다. <개정 2024. 9. 10.>\n\n  1. “주택”이란 「주택임대차보호법」 제2조에 따른 주거용 건물(공부상 주거용 건물이 아니라도 임대차계약 체결 당시 임대차목적물의 구조와 실질이 주거용 건물이고 임차인의 실제 용도가 주거용인 경우를 포함한다)을 말한다.\n\n  2. “임대인등”이란 임대인 또는 다음 각 목의 어느 하나에 해당하는 자를 말한다.\n\n    가. 임대인의 대리인, 그 밖에 임대인을 위하여 주택의 임대에 관하여 업무를 처리하는 자\n\n    나. 임대인의 의뢰를 받은 공인중개사(중개보조인을 포함한다)\n\n    다. 임대인을 위하여 임차인을 모집하는 자(그 피고용인을 포함한다)\n\n    라. 다수 임대인의 배후에 있는 동일인\n\n    마. 라목의 동일인이 지배하거나 경제적 이익을 공유하는 조직\n\n    바. 라목의 동일인이나 마목의 조직을 배후에 둔 다수의 임대인\n\n  3. “전세사기피해자”란 제3조의 요건을 모두 갖춘 자로서 제6조에 따른 전세사기피해지원위원회의 심의ㆍ의결을 거쳐 국토교통부장관이 결정한 임차인을 말한다.\n\n  4. “전세사기피해자등”이란 다음 각 목의 어느 하나에 해당하는 자를 말한다.\n\n

In [16]:
len(retrieved_docs)

4